### Coleta usando a API do twitter

In [57]:
import tweepy  # pip3 install tweepy
import sys
import os
import json
import time
import csv
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
import threading
import re

In [64]:
def criaJsonSaida(json_input):
    return {
        "id": json_input["id"],
        "data_criacao": json_input["created_at"],
        "tweet": json_input["text"].lower(),
        "nome_usuario": json_input["user"]["name"],
        "e_sobre_a_empresa" : None
    }

def Collect(api, searchQuery, maxTweets, tweetsPerQry, arquivo):
    tweetsColetados = {
        "tweets": []
    }
    # If results from a specific ID onwards are reqd, set since_id to that ID.
    # else default to no lower limit, go as far back as API allows
    sinceId = None

    # If results only below a specific ID are, set max_id to that ID.
    # else default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1

    tweetCount = 0
    print("Downloading max {0} tweets".format(maxTweets))

    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery,  # the search query string
                                            lang='pt-BR',
                                            count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery,
                                            lang='pt-BR',
                                            count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery,
                                            lang='pt-BR',
                                            count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=searchQuery,
                                            lang='pt-BR',
                                            count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets: 
#                 print("No more tweets found")
#                 break
                time.sleep(900) #15 min #tirar e descometar linhas acima
            else:
                for tweet in new_tweets:
                    tweetJson = criaJsonSaida(tweet._json)
                    tweetsColetados["tweets"] += [tweetJson]
                tweetCount += len(new_tweets)
                max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            print("some error : " + str(e))
            break
    with open(arquivo, 'w') as f:
        f.write(json.dumps(tweetsColetados, indent=2))
        f.close()
    print ("Downloaded {0} tweets, Saved to {1}\n".format(tweetCount, arquivo))

In [54]:
api_key = ''
api_secret_key = ''

auth = tweepy.AppAuthHandler(api_key, api_secret_key)

api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 # Whether or not to print a notification when Tweepy is waiting for rate limits to replenish
                 wait_on_rate_limit_notify=True
                 )
if (not api):
    print ("Can't Authenticate")

consulta = 'sadia'  # this is what we're searching for
max_tweets = 100000  # Some arbitrary large number
max_tweets_por_consulta = 15  # this is the max the API permits
arquivo_com_tweetes_coletados = 'tweets.json'  # We'll store the tweets in a text file.

Collect(api, consulta, 100, max_tweets_por_consulta, arquivo_com_tweetes_coletados)

### Gerando csv com os tweets a serem rotulados

In [ ]:
with open(arquivo_com_twittes_coletados, 'r') as f:
    dic_tweets = json.loads(f.read())
    tweetsDF = pd.read_json(json.dumps(dic_tweets['tweets']))
nome_arquivo_com_tweetes_rotulados = 'tweets.csv'
tweetsDF.to_csv(nome_arquivo_com_tweetes_rotulados, index_label='id', quoting=csv.QUOTE_ALL)

### Lendo o csv com os tweets rotulados

In [9]:
nome_arquivo_com_tweetes_rotulados = 'tweetsTestHistoy.csv' 

In [11]:
tweets_rotuladosDF = pd.read_csv(nome_arquivo_com_tweetes_rotulados, index_col='id')

### Fazendo um pré-processamento nos textos dos tweets

### Identificando os brigramas e trigramas encontrados nos twittes

In [67]:
tknzr = TweetTokenizer()

ngram = {}
for indice, linha in tweets_rotuladosDF.iterrows() :
    tokens = tknzr.tokenize(linha[tweets_rotuladosDF.tweet.name])
    for count in range(len(tokens)):
        if(tokens[count] == 'sadia'):
            bigram = (tokens[count - 1], tokens[count])
            trigram = (tokens[count - 2], tokens[count - 1], tokens[count])
            
            if ngram.get(bigram) == None:
                ngram[bigram] = {"frequencia" : 1, "id_tweets" : [indice]}
            else:
                valor = ngram.get(bigram)
                valor["frequencia"] += 1
                valor["id_tweets"] += [indice]
            if ngram.get(trigram) == None:
                ngram[trigram] = {"frequencia" : 1, "id_tweets" : [indice]}
            else:
                valor = ngram.get(trigram)
                valor["frequencia"] += 1
                valor["id_tweets"] += [indice]
                

In [68]:
ngram

{('&', 'sadia'): {'frequencia': 1, 'id_tweets': [10]},
 ('aberta', 'e', 'sadia'): {'frequencia': 1, 'id_tweets': [5]},
 ('brincadeirinha', 'sadia'): {'frequencia': 1, 'id_tweets': [0]},
 ('conversa', 'sadia'): {'frequencia': 1, 'id_tweets': [4]},
 ('e', 'sadia'): {'frequencia': 1, 'id_tweets': [5]},
 ('jovem', '&', 'sadia'): {'frequencia': 1, 'id_tweets': [10]},
 ('sociedade', 'sadia'): {'frequencia': 2, 'id_tweets': [1, 7]},
 ('uma', 'brincadeirinha', 'sadia'): {'frequencia': 1, 'id_tweets': [0]},
 ('uma', 'conversa', 'sadia'): {'frequencia': 1, 'id_tweets': [4]},
 ('uma', 'sociedade', 'sadia'): {'frequencia': 2, 'id_tweets': [1, 7]}}